In [ ]:
import tensorflow as tf
import pandas as pd
import shutil
from tensorboard.plugins.hparams import api as hp

import numpy as np
from tqdm import tqdm
import logging
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
import seaborn as sns
import matplotlib.pyplot as plt
from numpy.polynomial import Polynomial

from utilities import get_npz_data, get_project_data, get_bci_iii_data, get_bci_iii_data

tf.random.set_seed(333)
logger = logging.getLogger('1d-AX-LSTM')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

%load_ext tensorboard

# TensorBoard
------
#### Used to track the gridsearch and visualize the process of the system.

In [ ]:
%tensorboard --logdir LOG

# Data acquisition 
----
#### Showing different possibilities of acquiring data

In [ ]:
LOG_PATH = f'{os.getcwd()}/LOG/GridSearch_1dAX_LSTM'
HYBRID_DATA_PATH = f'{os.getcwd()}/Data/Hybrid'
BCI_IV_2a_DATA_PATH = f'{os.getcwd()}/Data/BCI'
BCI_III_IVa_DATA_PATH = f'{os.getcwd()}/Data/BCI III IVa'

L_EEG, L_TARGET = get_project_data(HYBRID_DATA_PATH, [2.0, 3.0], sec=3, offset=1)

if not os.path.exists(LOG_PATH):
    os.makedirs(LOG_PATH)

# Preprocessing
----
#### The first preprocessing step is the normalisation. The scheme for this is the follwing. Let us consider $\mathbf{X}_{\mathrm{EEG}}=[\mathbf{x}^{(1)},\ldots, \mathbf{x}^{(N)}]\in \mathbb{R}^{k \times N}$, where $k$ are the amount of channels and $N$ the observations. Then we take a vector $\mathbf{x}_{k}=[\mathrm{x}^{(1)}_{k},\ldots, \mathrm{x}^{(N)}_{k}]$, calculate its mean $\mu_{k}$, standard deviation $\sigma_{k}$ and normalise it as $\mathbf{\bar{x}}_{k}=[\frac{\mathrm{x}^{(1)}_{k}-\mu_{k}}{\sigma_{k}},\ldots, \frac{\mathrm{x}^{(N)}_{k}-\mu_{k}}{\sigma_{k}}]$. Repeating this for each channel, we get $\mathbf{\bar{X}}_{\mathrm{EEG}}$.

In [ ]:
def _preprocessing(data):
    logger.info(f'Preprocessing data')
    normalized_data = np.empty(shape=data.shape)
    for idx, trial in enumerate(data):
        normalized_data[idx] = StandardScaler().fit_transform(X=trial)
    return normalized_data

# Feature Extraction
### One Dimension-Aggregate Approximation (1d-AX)
----
#### 1d-AX  can calculated by taking $\mathrm{\bar{x}}_{k}=[\mathrm{\bar{x}}^{(1)}_{k},\ldots, \mathrm{\bar{x}}^{(N)}_{k}]$ for each channel, and divide it into $m$ segements $\mathbf{y}_{k}^{(i)}$ of equal length $q=\frac{N}{m}$. Afterwards, we take a sampled version of $\mathbf{y}_{k}^{(i)}$ and apply linear regression to it.

In [ ]:
def _segment_data(data, seg_length):
    logger.info(f'Segmenting data')
    num_trials, num_samples, num_channels = data.shape
    assert num_samples % seg_length == 0
    num_segments = num_samples//seg_length
    seg_eeg_data = np.empty(shape=(num_trials, num_segments, seg_length, num_channels))

    for trial_idx in range(num_trials):
        for segment_idx in range(num_segments):
            lower_limit = segment_idx * seg_length
            upper_limit = lower_limit + seg_length
            seg_eeg_data[trial_idx, segment_idx] = data[trial_idx, lower_limit:upper_limit, :]

    return seg_eeg_data

In [ ]:
def _linear_regression(data):
    num_trials, num_segments, num_samples, num_channels = data.shape
    x = np.linspace(1, num_samples, num_samples)

    K = np.zeros(shape=(num_trials, num_channels, num_segments))
    A = np.zeros(shape=(num_trials, num_channels, num_segments))
    for trial_idx, trial in enumerate(data):
        for segment_idx, segment in enumerate(trial):
            for channel_idx in range(num_channels):
                (c1, c0) = np.polyfit(x, segment[:, channel_idx], deg=1)
                t_mean = np.mean(segment[:, channel_idx])
                K[trial_idx, channel_idx, segment_idx] = c1
                A[trial_idx, channel_idx, segment_idx] = c1 * t_mean + c0

    return K, A

# Channel Weighting
----
#### Following the 1d-AX step, we calculate spatial filters for dimensionality reduction. The idea is to gain filters $\mathbf{W_{\mathbf{K}}}$ and $\mathbf{W_{\mathbf{A}}}$, which reduce the dimensionality of $\mathbf{K}$ and $\mathbf{A}$ by taking $\mathbf{K}'=\mathbf{W_{\mathbf{K}}}\mathbf{K}$, as well as $\mathbf{A}'=\mathbf{W_{\mathbf{A}}}\mathbf{A}$.

# LSTM and Softmax Regression
----
#### Subsequently, the output of the filters $\mathbf{K}'$, $\mathbf{A}'$ is fed into two LSTMs in parallel. Output of the LSTM is merged and fed into a Softmax Regression.

In [ ]:
def _fit(data, num_reduced_channels, LSTM_cells, num_classes):
    _, num_segments, num_channels = data.shape

    Input_K = tf.keras.Input(shape=(num_segments, num_channels), batch_size=None)
    Input_A = tf.keras.Input(shape=(num_segments, num_channels), batch_size=None)
    Input_K_f = tf.keras.layers.Dense(num_reduced_channels)(Input_K)
    Input_A_f = tf.keras.layers.Dense(num_reduced_channels)(Input_A)
    Input_K_f_norm = tf.keras.layers.BatchNormalization()(Input_K_f)
    Input_A_f_norm = tf.keras.layers.BatchNormalization()(Input_A_f)
    LSTM_K = tf.keras.layers.LSTM(LSTM_cells, dropout=0.6)(Input_K_f_norm)
    LSTM_A = tf.keras.layers.LSTM(LSTM_cells, dropout=0.6)(Input_A_f_norm)
    Merged_LSTM = tf.keras.layers.concatenate([LSTM_K, LSTM_A])
    Output = tf.keras.layers.Dense(num_classes, activation='softmax')(Merged_LSTM)

    return tf.keras.Model(inputs=[Input_K, Input_A], outputs=Output)

#### Training a model on a given set of training features and labels
_______

In [ ]:
def _train(K, A, target, spatial_filter_dim, LSTM_cells, num_classes, epochs, verbose, batch_size):
    model = _fit(K, spatial_filter_dim, LSTM_cells, num_classes)
    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.CategoricalCrossentropy()
    metrics = [tf.keras.metrics.CategoricalAccuracy()]
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)
    model.fit([K, A], target, epochs=epochs, verbose=verbose, batch_size=batch_size)
    return model

#### Evaluating a model on a given set of test data and labels
_______

In [ ]:
def evaluate(data, target, model, segment_length, batch_size=64):
    new_target = map_to_one_hot(target)
    normalized_data = _preprocessing(data)
    segmented_data = _segment_data(normalized_data, segment_length)
    K, A = _linear_regression(segmented_data)
    loss, acc = model.evaluate([K, A], new_target, batch_size=batch_size)
    return acc

#### Pipeline for training the model on a given data and labels
______

In [ ]:
def pipeline(data, target, segment_length, spatial_filter_dim, LSTM_cells, num_classes, epochs=500, verbose=1, batch_size=64):
    new_target = map_to_one_hot(target)
    normalized_data = _preprocessing(data)
    segmented_data = _segment_data(normalized_data, segment_length)
    K, A = _linear_regression(segmented_data)
    model = _train(K, A, new_target, spatial_filter_dim, LSTM_cells, num_classes, epochs, verbose, batch_size)
    return model

 #### Mapping class labels to a categorical variable.
 ______

In [ ]:
def map_to_one_hot(target):
    new_target = np.copy(target)
    unique = np.unique(target)
    new_labels = np.linspace(0, len(unique)-1, len(unique))
    mapping = dict(zip(unique, new_labels))
    for idx, label in enumerate(target):
        new_target[idx] = mapping[label]
    return tf.one_hot(new_target, depth=len(unique))

# Example of a possible training scheme
-------
#### Define parameters if interest, i.e. LSTM cells or dimension of the spatial filter.

In [ ]:
k_cross_val = 5
accs_shifted = list()
num_segments = 6
t_sec = 3
sampling_freq = 256
num_samples = t_sec * sampling_freq

spatial_filter_dim, LSTM_cells, num_classes =  6, 8, 2

for comb in [[0.0, 1.0], [0.0, 2.0], [0.0, 3.0], [1.0, 2.0], [2.0, 3.0], [2.0, 3.0]]:
    tmp = list()
    e_tmp1, t_tmp1 = get_project_data(HYBRID_DATA_PATH, comb, sec=t_sec, offset=1)
    e_tmp2, t_tmp2 = get_project_data(HYBRID_DATA_PATH, comb, sec=t_sec, offset=2)
    e_tmp3, t_tmp3 = get_project_data(HYBRID_DATA_PATH, comb, sec=t_sec, offset=3)
    EEG, TARGET = np.vstack([e_tmp1, e_tmp2, e_tmp3]), np.hstack([t_tmp1, t_tmp2, t_tmp3])
    for train_idx, test_idx in StratifiedKFold(n_splits=k_cross_val).split(np.zeros(shape=TARGET.shape), TARGET):
        EEG_TRAIN, TARGET_TRAIN = EEG[train_idx], TARGET[train_idx]
        EEG_TEST, TARGET_TEST = EEG[test_idx], TARGET[test_idx]

        model = pipeline(EEG_TRAIN, TARGET_TRAIN, num_samples//num_segments, spatial_filter_dim, LSTM_cells, num_classes, )
        acc = evaluate(EEG_TEST, TARGET_TEST, model, num_samples//num_segments)
        tmp.append(acc)
    accs_shifted.append(tmp)

# Hyperparameter Search
----------
#### Adapt the parameters _learning_rates_, _filter_dim_, _lstm_cells_, _num_segments_ and _epochs_ before starting the hypterparamter search.

In [ ]:
try:
    os.makedirs(LOG_PATH)
except FileExistsError:
    shutil.rmtree(LOG_PATH)
finally:
    os.makedirs(LOG_PATH)

learning_rates = [1e-3, 1e-2, 1e-1]
filter_dim = [3, 6, 9, 12]
lstm_cells = [1, 3, 5, 8, 10]
num_segments = [4, 6, 8]
epochs = [250, 500, 1000]

HP_LEARNING_RATE = hp.HParam('Learning Rate', hp.Discrete(learning_rates))
HP_FILTER_DIM = hp.HParam('Filter Dimension', hp.Discrete(filter_dim))
HP_LSTM_CELLS = hp.HParam('LSTM Cells', hp.Discrete(lstm_cells))
HP_NUM_SEGMENTS = hp.HParam('Number of Segments', hp.Discrete(num_segments))
HP_EPOCHS = hp.HParam('Epochs', hp.Discrete(epochs))
METRIC_ACCURACY = 'accuracy'

def run(path, hparams, train_data, train_labels, test_data, test_labels, fold_idx):
    with tf.summary.create_file_writer(path).as_default():
        hp.hparams(hparams)
        model = pipeline(train_data, train_labels,
                         segment_length=768//hparams[HP_NUM_SEGMENTS], 
                         spatial_filter_dim=hparams[HP_FILTER_DIM], 
                         LSTM_cells=hparams[HP_LSTM_CELLS], 
                         num_classes=4,
                         epochs=hparams[HP_EPOCHS])
        accuracy = evaluate(test_data, test_labels, model, 
                            segment_length=768//hparams[HP_NUM_SEGMENTS])

        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=fold_idx, description=f'Fold {fold_idx}')

In [ ]:
session_idx = 0
for lr_idx, lr in enumerate(HP_LEARNING_RATE.domain.values, 0):
    for fdim_idx, fdim in enumerate(HP_FILTER_DIM.domain.values, 0):
        for cells_idx, cells in enumerate(HP_LSTM_CELLS.domain.values, 0):
            for segment_idx, N in enumerate(HP_NUM_SEGMENTS.domain.values, 0):
                for epoch_idx, epoch_N in enumerate(HP_EPOCHS.domain.values, 0):
                    hparams = {
                        HP_LEARNING_RATE: lr,
                        HP_FILTER_DIM: fdim,
                        HP_LSTM_CELLS: cells,
                        HP_NUM_SEGMENTS: N,
                        HP_EPOCHS: epoch_N
                    }
                    
                    EEG, TARGET = get_project_data(HYBRID_DATA_PATH, [0.0, 1.0, 2.0, 3.0], sec=3, offset=1)
                    for fold_idx, (train_idx, test_idx) in enumerate(StratifiedKFold(n_splits=5).split(np.zeros(shape=TARGET.shape), TARGET)):
                        EEG_TRAIN, TARGET_TRAIN = EEG[train_idx], TARGET[train_idx]
                        EEG_TEST, TARGET_TEST = EEG[test_idx], TARGET[test_idx]

                        run(f'{LOG_PATH}/t-{session_idx}-fold-{fold_idx}', hparams, EEG_TRAIN, TARGET_TRAIN, EEG_TEST, TARGET_TEST, fold_idx)
                        
                    session_idx += 1